# 1. BASE

## 1-1) 필요 라이브러리 및 데이터 호출

In [ ]:
import pandas as pd
import datetime
from queue import PriorityQueue
from datetime import timedelta, datetime
import math
from itertools import combinations
from itertools import permutations
from collections import defaultdict

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## class 내부 변수 값을 볼 수 있는 함수
def print_class(class_instance):
    class_variables = dir(class_instance)
    class_variables = [var for var in class_variables if not var.startswith('__')]
    for var in class_variables:
        print(f"{var}: {getattr(class_instance, var)}")


## 오더 class 객체 생성 시 필요 함수
def find_overlapping_ranges(ranges, target_range):
    overlapping_ranges = []
    for start, end in ranges:
        if start <= target_range[1] and end >= target_range[0]:
            overlapping_start = max(start, target_range[0])
            overlapping_end = min(end, target_range[1])
            overlapping_ranges.append((overlapping_start, overlapping_end))

    return overlapping_ranges


## 오더 class 내 timewindow 변수 값 생성을 위한 함수
def time_window_return(start,end,group):
  result = []
  start = start*60.0
  end = end*60.0

  if start < end :
    for i in range(7):
      result.append((i*1440+start,i*1440+end))
  else:
    result.append((0.0,end))
    for i in range(5):
      result.append((i*1440+start,(i+1)*1440+end))
    result.append((6*1440+start,10080.0))

  if group <= 16:
    result2 = (group*360.0,group*360.0+4320.0)
  else:
    result2 = (group*360.0,10080.0)

  answer = find_overlapping_ranges(result,result2)
  return(answer)


def timestampToDateStr(time): # timestamp -> "년도-월-일 시:분"
    curDateTime = datetime(2023, 5, 1, 0, 0) # 고정 시작 시간
    time = curDateTime + timedelta(minutes = time)
    return time.strftime("%Y-%m-%d %H:%M")


def calculate_angle(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    angle_radians = math.atan2(dy, dx)
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees


def calculate_clockwise_rotation_angle(base_x, base_y, target_x, target_y):
    angle = calculate_angle(base_x, base_y, target_x, target_y)
    clockwise_rotation_angle = 90 - angle
    if clockwise_rotation_angle < 0:
        clockwise_rotation_angle += 360

    return clockwise_rotation_angle

In [ ]:
order = pd.read_csv("/content/drive/Shareddrives/CJ 대한통운 공모전/용탁/data/Orders.csv",encoding='cp949')
terminalCSV = pd.read_csv("/content/drive/Shareddrives/CJ 대한통운 공모전/용탁/data/Terminals.csv",encoding='cp949')
vehicle = pd.read_csv("/content/drive/Shareddrives/CJ 대한통운 공모전/용탁/data/Vehicles.csv")
distance = pd.read_csv("/content/drive/Shareddrives/CJ 대한통운 공모전/용탁/data/Distance.csv")

order = order.rename(columns = {'주문ID' : 'OrderID', '하차지_위도' : 'Des_Lat','하차지_경도' : 'Des_Lon','착지ID' : 'Des_ID','터미널ID' : 'Departure','하차가능시간_시작' : 'Unloaded_start','하차가능시간_종료' : 'Unloaded_end','하차작업시간(분)' : 'Timecost'})
order = order[["date","Group","OrderID","CBM","Departure","Des_ID","Des_Lat","Des_Lon","Unloaded_start","Unloaded_end","Timecost"]]
## 주문시작가능시간에 7:00을 07:00으로 바꾼후, 앞에 시간 만을 따서 int로 변환
order["Unloaded_start"] = order["Unloaded_start"].apply(lambda x: x.zfill(5) if len(x) == 4 else x).str[:2].astype(int)
order["Unloaded_end"] = order["Unloaded_end"].apply(lambda x: x.zfill(5) if len(x) == 4 else x).str[:2].astype(int)
order = pd.merge(order,distance, how='left', left_on=["Departure","Des_ID"], right_on=["origin","Destination"])
order.drop(labels=["origin","Destination"],axis=1, inplace=True)
order = order.rename(columns = {'Distance_km' : 'distance', 'Time_minute' : 'time'})

# date와 Group 컬럼을 참조하여 0~23으로 할당해주는 함수
def assign_group(data, start_date, num_days=6):
    unique_dates = pd.date_range(start=start_date, periods=num_days, freq='D')
    unique_groups = data['Group'].unique()

    counter = 0
    for date in unique_dates:
        for group in unique_groups:
            data.loc[(data['date'] == str(date.date())) & (data['Group'] == group), 'group'] = counter
            counter += 1
    return data

order = assign_group(order, '2023-05-01')
order['group'] = order['group'].astype(int)
order["timeWindow"]=None
order["angle"] = 0.0

ter_coordinate = terminalCSV[["ID","Origin_Lat","Origin_Lon"]].drop_duplicates()
ter_xy = dict()
for i in range(len(ter_coordinate)):
  ter_xy[ter_coordinate.iat[i,0]] = (ter_coordinate.iat[i,1],ter_coordinate.iat[i,2])

for i in range(len(order)):
  order.iat[i,14] = time_window_return(order.iat[i,8],order.iat[i,9],order.iat[i,13])
  base_x, base_y = ter_xy[order.iat[i,4]]
  order.iat[i,15] = calculate_clockwise_rotation_angle(base_x, base_y,order.iat[i,6],order.iat[i,7])

order = order.reset_index()
order.head(2)

,index,date,Group,OrderID,CBM,Departure,Des_ID,Des_Lat,Des_Lon,Unloaded_start,Unloaded_end,Timecost,distance,time,group,timeWindow,angle
0,0,2023-05-01,0,20230501B464859_001,27.893621,O_179,D_1,36.448483,127.40924,18,2,60,59.162692,48.164816,0,"[(0.0, 120.0), (1080.0, 1560.0), (2520.0, 3000...",310.162763
1,1,2023-05-01,1,20230501B464914_001,42.669815,O_673,D_1,36.448483,127.40924,18,2,60,183.779220,146.817218,1,"[(1080.0, 1560.0), (2520.0, 3000.0), (3960.0, ...",303.687695


In [ ]:
# order[["distance","time"]].describe()

#### A. 딕셔너리

- 목적지의 위도/경도 딕셔너리

In [ ]:
des_coordinate = order[["Des_ID","Des_Lat","Des_Lon"]].drop_duplicates()
des_xy = dict()

for i in range(len(des_coordinate)):
  des_xy[des_coordinate.iat[i,0]] = (des_coordinate.iat[i,1],des_coordinate.iat[i,2])

- "목적지"-"출발지" 거리/시간 딕셔너리

In [ ]:
'''Insoo = dict()
for i in distance.index:
  Insoo[distance.iat[i,0],distance.iat[i,1]] = (distance.iat[i,2],distance.iat[i,3])'''

'Insoo = dict()\nfor i in distance.index:\n  Insoo[distance.iat[i,0],distance.iat[i,1]] = (distance.iat[i,2],distance.iat[i,3])'

## 1-2) 클래스생성

- 최상위 객체 관리 리스트 및 딕셔너리

In [ ]:
# 각 주문 시점(key) 별로 주문 객체들(values)을 저장
orderbytime = {i: [] for i in range(24)}

# 터미널 객체 관리를 위한 딕셔너리
gTerminals = dict()     # 모든 터미널 객체. { "O_XXX": 터미널 객체 }

# 목적지에서 작업 완료한 차량 큐 / 목적지 -> 터미널로 복귀중인 차량 큐
unloadedComplete = PriorityQueue()  # (작업완료시간, 차량객체)
headToTer = PriorityQueue()         # (터미널도착시간, 복귀터미널객체, 차량객체)

# 전체 주문 객체를 관리하는 큐
deadlineList = PriorityQueue()        # (배송객체)

- 테이블 출력을 위한 데이터프레임 생성

In [ ]:
vehicleResults = pd.DataFrame(columns=['VehNum', 'Count', 'Volume', 'TravelDistance',
                                         'WorkTime', 'TravelTime', 'ServiceTime', 'WaitingTime',
                                         'TotalCost', 'FixedCost', 'VariableCost']) #  차량 결과 테이블
orderResults = pd.DataFrame(columns=['ORD_NO', 'VehicleID', 'Sequence', 'SiteCode', 'ArrivalTime', 'WaitingTime',
                                      'ServiceTime', 'DepartureTime', 'Delivered', 'Method']) # 주문 결과 테이블

- 클래스 생성 및 딕셔너리 리스트에 저장

In [ ]:
class Order: # 주문 상품
    def __init__(self,orderId,CBM,terminal,desID,timeWindow,timecost,group,angle):
        self.orderId = orderId;               # 주문 번호(ID)
        self.CBM = CBM                        # 적재 용량
        self.terminal = terminal              # 출발 터미널 ID
        self.desID = desID                    # 목적지(Des) ID
        self.timeWindow = timeWindow          # 하차 가능 시간대를 tuple로 담은 리스트
        self.timecost = 60                    # 하차작업시간(분)
        self.group = group                    # 주문 생성 시점, 0 ~ 23까지
        self.distance = Insoo[(self.terminal, self.desID)][0]              # 터미널-목적지 이동 거리
        self.time = Insoo[(self.terminal, self.desID)][1]                  # 터미널-목적지 걸리는 시간
        self.deadline = self.timeWindow[-1][-1]-self.time-0.000001        # 72시간 내 처리를 위한 출발 시각 마지노선 (2시 이전 도착이면 -> 1시59분59초)
        self.angle = angle
        self.deliveredMethod = 0

        self.vehId = None                     # 주문이 할당된 차량 ID
        self.seq = 'Null'                     # 배송 시퀀스
        self.siteCode = None                  # 도착지 ID -> desID를 이용하자.  => 필요?
        self.arrivalTime = 'Null'             # 도착시각
        self.waitingTime = 'Null'             # 대기시간 (72-4320분 미만-시간 내에 처리되어야 함)
        self.serviceTime = 'Null'             # 상하/하차시간
        self.departureTime = 'Null'           # 출발시각

        self.delivered = False                # 배송 완료 여부

    def __str__(self): # 디버깅용, 객체 정보 확인용
        return f"""{self.orderId}\n CBM:{self.CBM}\n terminal:{self.terminal}\n desID:{self.desID}\n timeWindow:{self.timeWindow}
 timecost:{self.timecost}\n group:{self.group}\n distance:{self.distance}\n time:{self.time}\n deadline:{self.deadline}
 vehId:{self.vehId}\n seq:{self.seq}\n siteCode:{self.siteCode}\n arrivalTime:{self.arrivalTime}
 waitingTime:{self.waitingTime}\n serviceTime:{self.serviceTime}\n departureTime:{self.departureTime}\n delivered:{self.delivered}
                """

    def __lt__(self, o):
        return self.deadline < o.deadline

    def updateForSTSD(self, vehId, seq):
        global nowClock
        self.vehId = vehId
        self.seq = seq
        self.delivered = True
        self.arrivalTime = nowClock + self.time      # 주문 처리 시점인 nowClock을 arrivalTime으로 설정
        self.departureTime = self.arrivalTime + 60.0
        self.serviceTime = 60
        self.deliveredMethod = 1

    def updateForSTDD(self, vehId, seq, plusTime):
        global nowClock
        self.vehId = vehId
        self.seq = seq
        self.delivered = True
        self.arrivalTime = nowClock + plusTime
        self.departureTime = self.arrivalTime + 60.0
        self.serviceTime = 60
        self.deliveredMethod = 2

    def updateForDTSD(self, vehId, seq, plusTime):
        global nowClock
        self.vehId = vehId
        self.seq = seq
        self.delivered = True
        self.arrivalTime = nowClock + plusTime
        self.departureTime = self.arrivalTime + 60.0
        self.serviceTime = 60
        self.deliveredMethod = 3

    def updateForDTDD(self, curVeh, seqNum, plusTime):
        global nowClock
        self.vehId = curVeh.id
        self.seq = seqNum
        self.siteCode = self.desID
        self.arrivalTime = nowClock + plusTime
        self.departureTime = self.arrivalTime + self.timecost
        self.delivered = True
        gTerminals[self.terminal]
        self.deliveredMethod = 4

    def updateForUrgent(self, vehId, seq):
        global nowClock
        self.vehId = vehId
        self.seq = seq
        self.delivered = True
        self.arrivalTime = nowClock + self.time
        self.departureTime = self.arrivalTime + 60.0
        self.serviceTime = 60
        self.deliveredMethod = 5

class Vehicle:
    def __init__(self,id, ton, max, sT, fixedCost, variableCost):
      ## 차량 결과 테이블 출력을 위한 변수 생성
        self.id = id                      # 차량 ID
        self.ton = ton                    # 차량 톤급(결과 테이블 작성용)
        self.maxCBM = max                 # 차량 최대 용량
        self.fixedCost = fixedCost        # 최초 운행비
        self.variableCost = variableCost  # 1km 당 운행비

        self.processed = list()           # 총 처리된 주문 객체 List
        self.totalDistance = 0            # 총 주행거리
        self.totalMoveTime = 0            # Total Move Time(총 이동시간)
        self.totalUnloadedTime = 0        # Total Unloaded Time(총 하역시간)
        self.totalWaitingTime = 0          # Total Waiting Time(총 대기시간)
        self.nowLoc = sT                  # 현재 위치
        self.terminalReturnTime = 0       # 터미널로 복귀한 시간 / active할 경우 차량 구매로 간주하여 해당 시각으로 초기화
        self.isActive = False

      ## 주문 결과 테이블 출력을 위한 변수 생성
        self.startTerminal = sT           # 출발 터미널 => 필요?

    def __str__(self):
      return f"""{self.id} - ton:{self.ton}\n maxCBM:{self.maxCBM}\n fixedCost:{self.fixedCost}\n variableCost:{self.variableCost}
 processed:{self.processed}\n tatalDistance:{self.totalDistance}\n totalMoveTime:{self.totalMoveTime}\n totalUnloadedTime:{self.totalUnloadedTime}
 totalWaitingTime:{self.totalWaitingTime}\n nowLoc:{self.nowLoc}\n isActive:{self.isActive}\n startTerminal:{self.startTerminal}\n"""

    def __lt__(self, o):
      return self.maxCBM < o.maxCBM

    def updateForSTSD(self, samedesID):

      global orderResults, nowClock, unloadedComplete       # nowClock : 주문할당시점

      curTerminal = gTerminals[self.nowLoc]                             # curTerminal : 현재 차량의 상주 터미널

      # 차량을 터미널 상차 후 출차시키는 과정
      if self.isActive:                                                 # 이미 구매한 차량인 경우
        curTerminal.activeVehicleList.remove(self)
        self.totalWaitingTime += nowClock - self.terminalReturnTime
      else:                                                             # 해당 터미널에서 차량을 새로 구매하는 경우
        self.isActive = True                                            # 차량을 활성화
        self.terminalReturnTime = nowClock                              # 차량을 활성화(구입)한 시기를 표기
        curTerminal.currVehNum += 1                                     # 터미널의 현재차량대수 update
        curTerminal.inactiveVehicleList.remove(self)

      # OrderResults의 1번행 작성
      orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': ['Null'], 'VehicleID': [self.id], 'Sequence': ['1'],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(self.terminalReturnTime)],
                                              'WaitingTime': [str(nowClock-self.terminalReturnTime)], 'ServiceTime': [' '],
                                              'DepartureTime': [timestampToDateStr(nowClock)], 'Delivered': ['Null'], 'Method': ['1']})],
                                             ignore_index=True)

      seqNum = 2                                                        # seqNum : orderResults의 sequence를 작성하기 위한 인덱스

      # 차량의 작업 완료 시점을 update
      unloadedComplete.put((nowClock + samedesID[0].time+60.0, self))      # 차량이 목적지로 향하고 있으므로 큐에 저장 (목적지 작업완료시간, 해당 차량)
      curTerminal.currVehNum -= 1                                       # 상주터미널에서 떠났기 때문에 현재 차량 수 -1

      # 목적지까지 이동하면서 차량 및 주문 정보 업데이트
      self.totalUnloadedTime += 60                                      # 총 하역시간 60분 추가(1번)

      for o in samedesID:                                               # 주문 하나씩 접근
        self.totalDistance += Insoo[(self.nowLoc, o.desID)][0]          # 총 이동거리 update (사실 1번만 해줘도 됨)
        self.totalMoveTime += Insoo[(self.nowLoc, o.desID)][1]          # 총 이동시간 update (사실 1번만 해줘도 됨)
        self.nowLoc = o.desID                                           # 현재 위치 update   (사실 1번만 해줘도 됨)
        # 주문 처리
        o.updateForSTSD(self.id, seqNum)                                # 해당 주문 객체의 내부 정보 update
        # 주문 처리 후, 주문 관련 자료 구조의 update
        self.processed.append(o)                                        # 해당 차량이 처리한 주문 리스트에 주문 객체 추가
        curTerminal.orderDict[o.desID].remove(o)                        # 주문이 처리되었기 때문에 해당 터미널의 주문 딕셔너리에서 제거

        orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': [o.orderId], 'VehicleID': [self.id], 'Sequence': [seqNum],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(o.arrivalTime)],
                                              'WaitingTime': ['0'], 'ServiceTime': ['60'],
                                              'DepartureTime': [timestampToDateStr(o.departureTime)], 'Delivered': ['Yes']})],
                                             ignore_index=True)
        seqNum += 1

      ## 터미널의 주문 감소
      curTerminal.orderNum -= len(samedesID)


    def updateForDTSD(self, path):

      global orderResults, nowClock, unloadedComplete

      curTerminal = gTerminals[self.nowLoc]                             # 차량 상주 터미널 지정

      # 차량을 터미널 상차 후 출차시키는 과정
      if self.isActive:                                                 # 이미 구매한 차량인 경우
        curTerminal.activeVehicleList.remove(self)
        self.totalWaitingTime += nowClock - self.terminalReturnTime
      else:                                                             # 해당 터미널에서 차량을 새로 구매하는 경우
        self.isActive = True                                            # 차량을 활성화
        self.terminalReturnTime = nowClock                              # 차량을 활성화(구입)한 시기를 표기
        curTerminal.currVehNum += 1                                     # 터미널의 현재차량대수 update
        curTerminal.inactiveVehicleList.remove(self)

      # OrderResults의 1번행 작성
      orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': ['Null'], 'VehicleID': [self.id], 'Sequence': ['1'],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(self.terminalReturnTime)],
                                              'WaitingTime': [str(nowClock-self.terminalReturnTime)], 'ServiceTime': [' '],
                                              'DepartureTime': [timestampToDateStr(nowClock)], 'Delivered': ['Null'], 'Method': ['2']})],
                                             ignore_index=True)

      # 차량 출차로 터미널의 차량 수 업데이트
      curTerminal.currVehNum -= 1

      plusTime = 0                                                      # 현재 시간(nowClock)으로부터 지난 시간을 측정
      seqNum = 2                                                        # seqNum : orderResults의 sequence를 작성하기 위한 인덱스

      ## 모든 상차 터미널 순회(주문 객체 처리x, 차량 객체 정보와 주문 결과 테이블의 상차지 방문만 update)
      for idx in range(len(path)-1):
        o = path[idx+1]

        self.totalDistance += Insoo[(self.nowLoc, o.terminal)][0]
        self.totalMoveTime += Insoo[(self.nowLoc, o.terminal)][1]
        plusTime += Insoo[(self.nowLoc, o.terminal)][1]
        self.nowLoc = o.terminal
        if (path[idx].terminal != o.terminal):                          # 직전 터미널과 동일하지 않다면 터미널 Sitecode 출력
          orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': ['Null'], 'VehicleID': [self.id], 'Sequence': [seqNum],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(nowClock+plusTime)],
                                              'WaitingTime': [' '], 'ServiceTime': [' '],
                                              'DepartureTime': [timestampToDateStr(nowClock+plusTime)], 'Delivered': ['Null']})],
                                             ignore_index=True)
          seqNum += 1

      ## 마지막 터미널에서 목적지로 이동
      o = path[-1]
      self.totalDistance += o.distance
      self.totalMoveTime += o.time
      plusTime += o.time
      self.nowLoc = o.desID

      ## 주문 객체 처리 & 주문 결과 테이블의 처리 결과 update
      for idx in range((len(path))):
        o = path[idx]
        self.processed.append(o)
        o.updateForDTSD(self.id, seqNum, plusTime)
        gTerminals[o.terminal].orderDict[o.desID].remove(o)

        gTerminals[o.terminal].orderNum -= 1                            # 터미널의 주문 감소

        orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': [o.orderId], 'VehicleID': [self.id], 'Sequence': [seqNum],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(o.arrivalTime)],
                                              'WaitingTime': ['0'], 'ServiceTime': ['60'],
                                              'DepartureTime': [timestampToDateStr(o.departureTime)], 'Delivered': ['Yes']})],
                                             ignore_index=True)
        seqNum += 1

      # 차량 객체 정보(하역 시간)update
      self.totalUnloadedTime += 60
      # 차량의 작업 완료 시점을 update
      unloadedComplete.put((nowClock+plusTime+60.0,self))


    def updateForSTDD(self, path):

      global orderResults, nowClock, unloadedComplete

      curTerminal = gTerminals[self.nowLoc]                                 # curTerminal : 현재 차량의 상주 터미널

      # 차량을 터미널 상차 후 출차시키는 과정
      if self.isActive:
        curTerminal.activeVehicleList.remove(self)                          # O(N)
        self.totalWaitingTime += nowClock - self.terminalReturnTime
      else:
        self.isActive = True
        self.terminalReturnTime = nowClock
        curTerminal.currVehNum += 1
        curTerminal.inactiveVehicleList.remove(self)

      # OrderResults의 1번행 작성
      orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': ['Null'], 'VehicleID': [self.id], 'Sequence': ['1'],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(self.terminalReturnTime)],
                                              'WaitingTime': [' '], 'ServiceTime': [' '],
                                              'DepartureTime': [timestampToDateStr(nowClock)], 'Delivered': ['Null'], 'Method': ['3']})],
                                             ignore_index=True)


      # 차량 출차로 터미널의 차량 수 업데이트
      curTerminal.currVehNum -= 1

      plusTime = 0                                                      # 현재 시간(nowClock)으로부터 지난 시간을 측정
      seqNum = 2                                                        # seqNum : orderResults의 sequence를 작성하기 위한 인덱스

      ## 마지막 주문 전까지 처리
      for idx in range(len(path)-1):
        o = path[idx]

        self.totalDistance += Insoo[(self.nowLoc, o.desID)][0]
        self.totalMoveTime += Insoo[(self.nowLoc, o.desID)][1]
        plusTime += Insoo[(self.nowLoc, o.desID)][1]
        self.nowLoc = o.desID

        #주문 처리 후, 주문 관련 자료 구조의 update
        if (o.desID != path[idx+1].desID):                              # 현재 목적지가 다음 방문 목적지와 다르다면
          self.totalUnloadedTime += 60
          o.updateForSTDD(self.id, seqNum, plusTime)
          plusTime += 60
        else:                                                           # 현재 목적지가 다음 방문 목적지와 같다면
          self.totalUnloadedTime += 0
          o.updateForSTDD(self.id, seqNum, plusTime)
          plusTime += 0

        self.processed.append(o)
        curTerminal.orderDict[o.desID].remove(o)

        orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': [o.orderId], 'VehicleID': [self.id], 'Sequence': [seqNum],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(o.arrivalTime)],
                                              'WaitingTime': ['0'], 'ServiceTime': ['60'],
                                              'DepartureTime': [timestampToDateStr(o.departureTime)], 'Delivered': ['Yes']})],
                                             ignore_index=True)

        seqNum += 1

      ## 마지막 주문처리
      o = path[-1]

      self.totalDistance += Insoo[(self.nowLoc, o.desID)][0]
      self.totalMoveTime += Insoo[(self.nowLoc, o.desID)][1]
      plusTime += Insoo[(self.nowLoc, o.desID)][1]
      self.nowLoc = o.desID

      self.totalUnloadedTime += 60
      o.updateForSTDD(self.id, seqNum, plusTime)
      self.processed.append(o)

      curTerminal.orderDict[o.desID].remove(o)

      orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': [o.orderId], 'VehicleID': [self.id], 'Sequence': [seqNum],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(o.arrivalTime)],
                                              'WaitingTime': ['0'], 'ServiceTime': ['60'],
                                              'DepartureTime': [timestampToDateStr(o.departureTime)], 'Delivered': ['Yes']})],
                                             ignore_index=True)
      # 터미널의 주문 감소
      curTerminal.orderNum -= len(path)
      # 차량의 작업 완료 시점을 update
      unloadedComplete.put((nowClock+plusTime+60.0,self))


    def updateForDTDD(self, path): # path는 O-O-D-D 순\

      global orderResults, nowClock, unloadedComplete

      curTerminal = gTerminals[self.nowLoc]           # 상주 터미널 업데이트
      # 차량 활성화
      if self.isActive:
        curTerminal.activeVehicleList.remove(self)
        self.totalWaitingTime += nowClock - self.terminalReturnTime
      else:
        self.isActive = True
        self.terminalReturnTime = nowClock
        curTerminal.currVehNum += 1
        curTerminal.inactiveVehicleList.remove(self)

      # 차량 출차로 터미널의 차량 수 업데이트
      curTerminal.currVehNum -= 1

      plusTime = 0
      seqNum = 1

      # 터미널 순회에 대한 테이블 출력
      for i in range(2):
        if (i == 0):
          orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': ['Null'], 'VehicleID': [self.id], 'Sequence': [seqNum],\
                                              'SiteCode': [path[i].id], 'ArrivalTime': [timestampToDateStr(self.terminalReturnTime + plusTime)],\
                                              'WaitingTime': [str(nowClock-self.terminalReturnTime)], 'ServiceTime': [' '],\
                                              'DepartureTime': [timestampToDateStr(nowClock + plusTime)], 'Delivered': ['Null'], 'Method': ['4']})],\
                                             ignore_index=True)
          seqNum += 1
        elif(path[0].id != path[1].id):
          orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': ['Null'], 'VehicleID': [self.id], 'Sequence': [seqNum],\
                                              'SiteCode': [path[i].id], 'ArrivalTime': [timestampToDateStr(self.terminalReturnTime + plusTime)],\
                                              'WaitingTime': [' '], 'ServiceTime': [' '],\
                                              'DepartureTime': [timestampToDateStr(nowClock + plusTime)], 'Delivered': ['Null']})],\
                                             ignore_index=True)
          seqNum += 1

        # O1(터미널 객체)->O2(터미널 객체)
        if(i == 0):
          plusTime += Insoo[(path[i].id, path[i + 1].id)][1]
          self.totalMoveTime += plusTime
          self.totalDistance += Insoo[(path[i].id, path[i + 1].id)][0]


      # 주문 처리에 대한 Update
      # O2(터미널 객체)->D1(주문 객체)
      plusTime += Insoo[(path[1].id, path[2].desID)][1]                       # 목적지 D1도착
      self.totalMoveTime += Insoo[(path[1].id, path[2].desID)][1]
      self.totalDistance += Insoo[(path[1].id, path[2].desID)][0]

      path[2].updateForDTDD(self, seqNum, plusTime)                           # 주문 객체 D1처리

      self.totalUnloadedTime += 60                                            # 하역 시간 update
      self.processed.append(path[2])
      gTerminals[path[2].terminal].orderDict[path[2].desID].remove(path[2])

      orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': [path[2].orderId], 'VehicleID': [self.id], 'Sequence': [seqNum],\
                                              'SiteCode': [path[2].desID], 'ArrivalTime': [timestampToDateStr(path[2].arrivalTime)],\
                                              'WaitingTime': ['0'], 'ServiceTime': ['60'],\
                                              'DepartureTime': [timestampToDateStr(path[2].departureTime)], 'Delivered': ['Yes']})],\
                                             ignore_index=True)

      seqNum += 1

      # D1(터미널 객체)->D2(주문 객체)
      plusTime += 60 + Insoo[(path[2].desID, path[3].desID)][1]               # 목적지 D2도착
      self.totalMoveTime += Insoo[(path[2].desID, path[3].desID)][1]
      self.totalDistance += Insoo[(path[2].desID, path[3].desID)][0]

      path[3].updateForDTDD(self, seqNum, plusTime)                           # 주문 객체 D2처리

      self.totalUnloadedTime += 60                                            # 하역 시간 update
      self.processed.append(path[3])
      gTerminals[path[3].terminal].orderDict[path[3].desID].remove(path[3])

      orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': [path[3].orderId], 'VehicleID': [self.id], 'Sequence': [seqNum],\
                                              'SiteCode': [path[3].desID], 'ArrivalTime': [timestampToDateStr(path[3].arrivalTime)],\
                                              'WaitingTime': ['0'], 'ServiceTime': ['60'],\
                                              'DepartureTime': [timestampToDateStr(path[3].departureTime)], 'Delivered': ['Yes']})],\
                                             ignore_index=True)

      self.nowLoc = path[3].desID
      # 터미널의 주문 감소
      curTerminal.orderNum -= 2
      # 차량의 작업 완료 시점을 update
      unloadedComplete.put((nowClock+plusTime+60.0, self))


    def updateForUrgent(self, order):
      global orderResults, nowClock, unloadedComplete

      curTerminal = gTerminals[self.nowLoc]                                 # curTerminal : 현재 차량의 상주 터미널

      # 차량을 터미널 상차 후 출차시키는 과정
      if self.isActive:
        curTerminal.activeVehicleList.remove(self)                          # O(N)
        self.totalWaitingTime += nowClock - self.terminalReturnTime
      else:
        self.isActive = True
        self.terminalReturnTime = nowClock
        curTerminal.currVehNum += 1
        curTerminal.inactiveVehicleList.remove(self)

      # OrderResults의 1번행 작성
      orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': ['Null'], 'VehicleID': [self.id], 'Sequence': ['1'],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(self.terminalReturnTime)],
                                              'WaitingTime': [str(nowClock-self.terminalReturnTime)], 'ServiceTime': [' '],
                                              'DepartureTime': [timestampToDateStr(nowClock)], 'Delivered': ['Null'], 'Method': ['5']})],
                                             ignore_index=True)


      # 차량 출차로 터미널의 차량 수 업데이트
      curTerminal.currVehNum -= 1

      # 차량 해당 목적지로 이동
      self.totalDistance += order.distance
      self.totalMoveTime += order.time
      self.nowLoc = order.desID
      # 주문처리
      self.totalUnloadedTime += 60
      order.updateForUrgent(self.id, 2)
      self.processed.append(order)

      curTerminal.orderDict[order.desID].remove(order)

      orderResults = pd.concat([orderResults, pd.DataFrame({'ORD_NO': [order.orderId], 'VehicleID': [self.id], 'Sequence': [2],
                                              'SiteCode': [self.nowLoc], 'ArrivalTime': [timestampToDateStr(order.arrivalTime)],
                                              'WaitingTime': ['0'], 'ServiceTime': ['60'],
                                              'DepartureTime': [timestampToDateStr(order.departureTime)], 'Delivered': ['Yes']})],
                                             ignore_index=True)

      curTerminal.orderNum -= 1

      # 차량의 작업 완료 시점을 update
      unloadedComplete.put((nowClock + order.time + 60.0,self))             # 차량이 목적지로 향하고 있으므로 큐에 저장 (목적지 작업완료시간, 해당 차량)

    def updateForStarvation(self, top1Ter):
      global nowClock, unloadedComplete
      # 터미널 정보 업데이트
      curTerminal = gTerminals[self.nowLoc]


      if self.isActive:
        curTerminal.activeVehicleList.remove(self)
        self.totalWaitingTime += nowClock - self.terminalReturnTime
      else:
        self.isActive = True
        self.terminalReturnTime = nowClock
        curTerminal.currVehNum += 1
        curTerminal.inactiveVehicleList.remove(self)

      curTerminal.currVehNum -= 1

      top1Ter.expVehNum += 1

      # 차량 정보 업데이트
      self.totalDistance += Insoo[(self.nowLoc, top1Ter.id)][0]
      self.totalMoveTime += Insoo[(self.nowLoc, top1Ter.id)][1]

      headToTer.put((nowClock + Insoo[(self.nowLoc, top1Ter.id)][1], top1Ter, self))

'''
        self.totalDistance = 0            # 총 주행거리
        self.totalMoveTime = 0            # Total Move Time(총 이동시간)
        self.totalUnloadedTime = 0        # Total Unloaded Time(총 하역시간)
        self.totalWaitingTime = 0          # Total Waiting Time(총 대기시간)
        self.nowLoc = sT                  # 현재 위치
        self.terminalReturnTime = 0
'''

class Terminal:
    def __init__(self, id, latitude, longitude):
        self.id = id
        self.lat = latitude
        self.lon = longitude
        self.orderDict = dict()   # 터미널에서 처리해야 하는 주문 dict {"D_XX", [주문...]}
        self.orderNum = 0
        self.activeVehicleList = list() # 해당 터미널에 현재 사용 가능한 활성 차량 모음.
        self.inactiveVehicleList = list() # 해당 터미널의 비활성 차량 모음.
        self.currVehNum = 0       # 현재 차량 대수
        self.expVehNum = 0        # 도착 예정 차량

    def __lt__(self, other):
      return self.id < other.id

- Order 클래스 생성

In [ ]:
# 각 주문 시점(key) 별로 주문 객체들(values)을 저장
orderbytime = {i: [] for i in range(24)}

for i in range(1,25):
    globals()['T'+str(i)] = order[(order["date"]==f"2023-05-0{(i+3)//4}")&(order["Group"]==(i+3)%4)]

for i in range(1,25):
  temp = globals()['T'+str(i)].copy()
  for j in temp.index:
    globals()['O'+str(temp.at[j,"index"])] = Order(temp.at[j,"OrderID"], temp.at[j,"CBM"], temp.at[j,"Departure"], temp.at[j,"Des_ID"], temp.at[j,"timeWindow"],temp.at[j,"Timecost"], temp.at[j,"group"], temp.at[j,"angle"])
    orderbytime[(i-1)].append(globals()['O'+str(temp.at[j,"index"])])

- Terminal 클래스 생성

In [ ]:
for _, row in terminalCSV.iterrows():
  gTerminals[row['ID']] = Terminal(row['ID'], row['Origin_Lat'], row['Origin_Lon'])

- Vehicle 클래스 생성

In [ ]:
vehicle.sort_values(by='MaxCapaCBM', ascending=False, inplace=True)

for idx, row in vehicle.iterrows():
  globals()[row['VehNum']] = Vehicle(row['VehNum'], row['VehTon'], row['MaxCapaCBM'], row['StartCenter'], row['FixedCost'], row['VariableCost'])
  gTerminals[globals()[row['VehNum']].startTerminal].inactiveVehicleList.append(globals()[row['VehNum']])

# 2. 필요 함수 생성

## 2-1) 내장 함수

In [ ]:
# time window사이에서 겹치는 범위가 있는 지 확인하는 함수(SUB함수)
def is_overlap(ranges, target_range):
    for r in ranges:
        if r[0] < target_range[1] and target_range[0] <= r[1]:
            return True
    return False

#리스트 안의 클래스들의 특정 변수 값을 전부 더해주는 함수
def sum_variable_in_class_list(class_list, variable_name):
    total = 0
    for obj in class_list:
        if hasattr(obj, variable_name):
            total += getattr(obj, variable_name)
    return total
### ex) sum_variable_in_class_list(List,'CBM')


# time window 안에 value가 있는지 확인하는 함수
def is_within_ranges(value, tuple_list):
    for start, end in tuple_list:
        if start <= value < end:
            return True
    return False
'''
### 주어진 값과 튜플 리스트
value = 25.3
tuple_list = [(0.0, 120.0), (1080.0, 1560.0), (2520.0, 3000.0), (3960.0, 4320.0)]
### 함수 호출
result = is_within_ranges(value, tuple_list)
print(result)  # 출력: True
'''

# 터미널 객체 내의 orderDict를 리스트로 바꾸기 위한 함수
def flatten_dict_values(dictionary):
    flattened_list = []
    for value_list in dictionary.values():
        flattened_list.extend(value_list)
    return flattened_list

## 주문 객체 리스트를 처리 단위 형태의 딕셔너리로 변환
def classifyOrder1(order) :  # 1번에 적합한 주문 dictionary로 리턴 {(desID, terminal) : [O1, O2, ...], ...}
  result = sorted(order, key = lambda x: (x.desID, x.terminal)) # desID, terminal로 정렬

  grouped_result = defaultdict(list)
  for item in result: # dictionary 생성
    grouped_result[(item.desID, item.terminal)].append(item)

  grouped_result = {key: value for key, value in grouped_result.items() if len(value) >= 2} # 각 value값인 list 원소가 2 이상인 것만

  return grouped_result

def classifyOrder2(order) :  # 2번에 적합한 주문 dictionary로 리턴 { desID : [O1, O2, ...], ...}
  result = sorted(order, key = lambda x: x.desID ) # desID, terminal로 정렬

  grouped_result = defaultdict(list)
  for item in result: # dictionary 생성
    grouped_result[item.desID].append(item)

  grouped_result = {key: value for key, value in grouped_result.items() if len(value) >= 2} # 각 value값인 list 원소가 2 이상인 것만

  return grouped_result

def classifyOrder3(order) : # 3번에 적합한 주문 dictionary로 리턴 { terminal : [O1, O2, ...], ...}
  result = sorted(order, key = lambda x: x.terminal ) # desID, terminal로 정렬

  grouped_result = defaultdict(list)

  for item in result: # dictionary 생성
    grouped_result[item.terminal].append(item)

  grouped_result = {key: value for key, value in grouped_result.items() if len(value) >= 2} # 각 value값인 list 원소가 2 이상인 것만

  return grouped_result

- 1번 알고리즘을 위한 내부 함수

In [ ]:
def commonDistribute(data, curCBM): # data는 공동배송지 list
  data.sort(key=lambda x: x.CBM)
  curList = list()
  totalCBM = 0

  while len(data) != 0:
    start = 0
    end = len(data) - 1

    while start <= end:
      mid = (start + end) // 2
      if data[mid].CBM <= curCBM:
        start = mid + 1
      elif data[mid].CBM > curCBM:
        end = mid - 1
    if end != -1:
      curList.append(data[end])
      totalCBM += data[end].CBM
      curCBM -= data[end].CBM
      data.pop(end)
    else: break

  return curList

- 2, 3번 알고리즘을 위한 내부 함수

In [ ]:
# 주문 객체 리스트에서 현재시점으로부터 (목적지 이동 후, 그로부터 6시간이라는 시간 뒤)동안 해당 객체에 있을 때 배송 가능한 주문들을 출력해주는 함수
def PossibleOrder(timenow,orderlist): # time은 현재시각("00:00") / order는 주문 데이터 / wt는 기다릴 수 있는 최대 waiting time-------------------------
  ## median = get_proper_waitingtime(orderlist)
  possible = list()

  for i in orderlist:
    tw = (timenow+i.time, timenow+i.time+600.0)  # Orderlist안에 있는 객체마다 그에 맞는 타임 윈도우를 계산
    if (is_overlap(i.timeWindow,tw)):
      possible.append(i)

  return (possible)

# 모든 경로 조합
def PossibleCombination(OrderList):
  print("possibleCombination...")
  result = []
  n = 4
  while(n >= 2):
    result.append(list(combinations(OrderList, n)))
    n -= 1
  return result

# 모든 경로 조합 중 적재량을 고려한 조합 선별----------------------------------
def CapacitiyLimitCombination(OrderList,capacity = 55):
  print("CapacitiyLimitCombination...")
  allcombinationList = PossibleCombination(OrderList)
  result = []
  for i in allcombinationList:
    for j in i:
      if(sum_variable_in_class_list(j,'CBM') <= capacity):
        result.append(j)
  return result


# 현재 주어진 객체들의 조합 경로가 가능한지 True, False를 반환해주는 함수1 / 터미널이 같지만 목적지가 다를 때만 사용 가능
def RoutePossible_Check3(now,List):
  t = now                                                        ## 현재 시간을 t에 복사
  n = len(List)

  if (is_within_ranges(t + List[0].time,List[0].timeWindow)):    ## 현재 시점 t로부터 이동시간을 더했을 때 timewindow안에 들어간다면
    t += List[0].time
  else:
    return False

  for i in range(1,n):                                           ## List는 CBM의 도합이 55를 넘지 않는 Order class를 담은 리스트, [Order1,Order3,Order2]
    movetime = Insoo[List[i-1].desID,List[i].desID][1]
    if(List[i-1].desID != List[i].desID):
      t += movetime + 60
      if not(is_within_ranges(t,List[i].timeWindow)): return False

  return True

# 현재 주어진 객체들의 조합 경로가 가능한지 True, False를 반환해주는 함수2 / 목적지가 같고 터미널이 다를 때만
def RoutePossible_Check2(now,List):
  t = now                                                         ## 현재 시간을 t에 복사
  n = len(List)

  for i in range(0,n-1):                                          ## List는 CBM의 도합이 55를 넘지 않는 Order class를 담은 리스트, [Order1,Order3,Order2]
    movetime = Insoo[List[i].terminal,List[i+1].terminal][1]
    t += movetime

  if (is_within_ranges(t + List[n-1].time,List[n-1].timeWindow)):
    return True
  else:
    return False

# TimeWindow와 CBM을 모두 고려하여 각각의 경로 산출
def PossibleRoute3(OrderList,capacity=55):
  result = CapacitiyLimitCombination(OrderList,capacity)
  answer = []
  for i in result:
    permutations_list = list(permutations(i))
    for j in permutations_list:
      if(RoutePossible_Check3(0.0,j)):
        answer.append(j)
  print("가능한 경로 조합3")
  print(answer)
  return answer

def PossibleRoute2(OrderList,capacity=55):
  result = CapacitiyLimitCombination(OrderList,capacity)
  answer = []
  for i in result:
    permutations_list = list(permutations(i))
    for j in permutations_list:
      if(RoutePossible_Check2(0.0,j)):
        answer.append(j)
  print("가능한 경로 조합2")
  print(answer)
  return answer

In [ ]:
def check_duplicates(list1, tuple2):
  combined_tuple = tuple2
  for tup in list1 :
    combined_tuple += tup
  unique_elements = set(combined_tuple)

  return len(combined_tuple) != len(unique_elements) ## 중복값이 있는 지에 따라 True/Fasle로 반환


def check_len(routelist) : # 리스트 내에 있는 튜플 원소의 수 구하기
  length = 0
  for route in routelist :
    length += len(route)
  return length


def Calcul_Distance2(List): #DiffTerSameDes에 대한 각 튜플에 대한 거리
  distance = 0
  for tup in List:
    distance += tup[0].distance
    for i in range(1, len(tup) - 1) :
      distance += Insoo[tup[i].terminal, tup[i + 1].terminal][0]
  return distance


def Calcul_Distance3(List): #SameTerDiffDes에 대한 각 튜플에 대한 거리
  distance = 0
  for tup in List:
    distance += tup[0].distance
    for i in range(1, len(tup) - 1) :
      distance += Insoo[tup[i].desID, tup[i + 1].desID][0]
  return distance


'''def FindMostLeastCost(List, route, n) : # 함수 호출 FindMostLeastCost(리스트, 1차 조합된 경로들, n = 2(DiffTerSameDes), 3(SameTerDiffDes)), return : 최적경로 list
  global nowClock
  bestRoute = []
  mostShort = 9999999999999
  distance = 0
  i = 0
  for order in List : # len = 1인 튜플들 추가 ex. (1), (2), ..
    route.append((order,))

  for i in range(len(route)) : # 중복이 없는 튜플들을
    result = []
    result.append(route[i])
    for j in range(i + 1, len(route)):

      if check_duplicates(result, route[j]) == False:
        result.append(route[j])

        if check_len(result) == len(List) :
          if n == 2 :
            distance = Calcul_Distance2(result)
          elif n == 3 :
            distance = Calcul_Distance3(result)
          else : distance = 0

          if mostShort > distance :
            mostShort = distance
            bestRoute = result

          result = []
          result.append(route[i])

        continue

  length = len(bestRoute)
  for i in range(length-1,0, -1):
    if len(bestRoute[i]) == 1 and not is_within_ranges(nowClock+bestRoute[i][0].time,bestRoute[i][0].timeWindow):
      print("원소가 1개였던 것:", bestRoute[i])
      bestRoute.remove(bestRoute[i])
  return bestRoute'''

def FindMostLeastCost(orderList, possibleRoute, n):
  cnt = 0
  bestRoute = list()
  visited = set()
  for order in orderList : # len = 1인 튜플들 추가 ex. (1), (2), ..
    possibleRoute.append((order,))
  possibleRoute.sort(key=lambda x: (len(x), sum(o.CBM for o in x)), reverse=True)
  for pr in possibleRoute:
    cnt += 1
    sw = False
    for r in pr:
      if r in visited:
        sw = True
        break
    if sw: continue

    if (len(pr) > 1) or (len(pr) == 1 and is_within_ranges(nowClock+pr[0].time, pr[0].timeWindow)):
      bestRoute.append(pr)
    for r in pr:
      visited.add(r)
  return bestRoute

## 2-2) 1/2/3/4번 알고리즘

### 1번 알고리즘 함수

우선 큰 물건부터 큰 차에 싣는다. 다만 단건 배송이나 CBM의 합이 20이 넘지 않는 경우에는 유보한다.

In [ ]:
def sameTerSameDest(): # 객체 변수에 맞게 수정 orders 제거, terminal이 가지고 있음. 단건 반드시 유보. 해당 터미널에 차량이 없는 경우를 고려하지 않고 있다.

  global nowClock

  for terminalId, terminal in gTerminals.items():
    print(nowClock)
    #print("----------------------------------\n현재 터미널:", terminalId)
    if terminal.orderNum == 0:
      # print("주문 없음!")
      continue

    if len(terminal.activeVehicleList) == 0 and len(terminal.inactiveVehicleList) == 0:
      print("아예 터미널에 차량 자체가 없음! 넌 큰일 났어!")
      continue

    for dest, ordersList in terminal.orderDict.items():
      if len(ordersList) == 0 or not is_within_ranges(nowClock+ordersList[0].time,ordersList[0].timeWindow):continue
      activeVehicleList = terminal.activeVehicleList
      inactiveVehicleList = terminal.inactiveVehicleList

      i = 0
      j = 0

      while((i < len(activeVehicleList) or j < len(inactiveVehicleList)) and len(ordersList) > 0):

        curVeh = None
        if i != len(activeVehicleList):
          curVeh = activeVehicleList[i]       ## curVeh = 주문을 처리해줄 구세주 차량
          i += 1
          #print("\n선택된 차량:", curVeh.id)
          sameDest = commonDistribute(ordersList.copy(), curVeh.maxCBM)
          if len(sameDest) <= 1: continue
          curVeh.updateForSTSD(sameDest)
          i -= 1

        elif j != len(inactiveVehicleList):
          curVeh = inactiveVehicleList[j]
          j += 1
          #print("\n선택된 차량:", curVeh.id)
          sameDest = commonDistribute(ordersList.copy(), curVeh.maxCBM)
          if len(sameDest) <= 1: continue
          curVeh.updateForSTSD(sameDest)
          j -= 1


### 2번 알고리즘 함수

In [ ]:
def difTerSameDest():


  ## 디버깅을 위한 함수 + 터미널 별로 구분되어 있는 주문 하나로 모으기
  totalOrderList = list()

  for terminalId, terminal in gTerminals.items():

    print("----------------------------------\n현재 터미널:", terminalId)
    if terminal.orderNum == 0:
      print("주문 없음!")
      continue

    totalOrderList.extend(flatten_dict_values(terminal.orderDict))                                  # 현재 터미널에 차량이 없어도 타 터미널의 출발 차량에 의해 처리될 가능성이 존재

    if len(terminal.activeVehicleList) == 0 and len(terminal.inactiveVehicleList) == 0:
      print("아예 터미널에 차량 자체가 없음! 넌 큰일 났어!")
      continue

  ## 두 건 이상의 묶음 배송 건에 대해서만 목적지 별로 정리한 Dictonary생성
  orderBySameDes = classifyOrder2(totalOrderList)

  ## 각 Destination별로 알고리즘2 적용해서 주문 처리하기 {"D_17":[목적지가 같은 주문 객체],"D_16":[주문 객체]}
  for dest,orderList in orderBySameDes.items():
    print(nowClock)
    optimalRoutes = FindMostLeastCost(orderList,PossibleRoute2(orderList),2)
    print(f"2번알고리즘 실행,목적지:{dest}")
    print(optimalRoutes)
    for optimalRoute in optimalRoutes:                        # ex) optimalRoutes = [(1,3),(4,2)], optimalRoute = (1,3)

      firstTerminal = gTerminals[optimalRoute[0].terminal]    # (1,4,5)의 경로 가운데 차량을 가용하게 되는 첫 번째 터미널
      curVeh = None
      if len(firstTerminal.activeVehicleList) != 0 :
        curVeh = firstTerminal.activeVehicleList[0]           # curVeh = 주문을 처리해줄 차량
      elif len(firstTerminal.inactiveVehicleList) != 0:
        curVeh = firstTerminal.inactiveVehicleList[0]
      else:
        print("가용 가능한 차량이 한 대도 없습니다")
        continue

      if curVeh.maxCBM >= sum_variable_in_class_list(optimalRoute,"CBM"): # 선택된 차량으로 처리할 수 있으면 해당 주문을 처리
        curVeh.updateForDTSD(optimalRoute)
      else:                                                               # 큰 차량부터 선택하기 때문에, 현재 차량으로 처리가 안될 시 pass
        continue

### 3번 알고리즘 함수

In [ ]:
def sameTerDifDest():

  for terminalId, terminal in gTerminals.items():

    totalOrderList = list()

    print("----------------------------------\n현재 터미널:", terminalId)
    if terminal.orderNum == 0:
      print("주문 없음!")
      continue

    if len(terminal.activeVehicleList) == 0 and len(terminal.inactiveVehicleList) == 0:
      print("차량 없음!")
      continue

    for l in terminal.orderDict.values():
      for o in l:
        totalOrderList.append(o)

    processedOrders = FindMostLeastCost(totalOrderList,PossibleRoute3(totalOrderList),3)

    for path in processedOrders:
      print(nowClock)
      curVeh = None
      if len(terminal.activeVehicleList) != 0 :
        curVeh = terminal.activeVehicleList[0]           # curVeh = 주문을 처리해줄 차량
      elif len(terminal.inactiveVehicleList) != 0:
        curVeh = terminal.inactiveVehicleList[0]
      else:
        print("가용 가능한 차량이 한 대도 없습니다")
        continue

      if curVeh.maxCBM >= sum_variable_in_class_list(path,"CBM"): # 선택된 차량으로 처리할 수 있으면 해당 주문을 처리
        curVeh.updateForSTDD(path)
      else:                                                       # 큰 차량부터 선택하기 때문에, 현재 차량으로 처리가 안될 시 pass
        continue

### 4번 알고리즘 함수

In [ ]:
def CapacitiyLimitCombination4(OrderList,capacity = 55):
  result = []
  allcombinationList = list(combinations(OrderList, 2))
  for i in allcombinationList:
    if(sum_variable_in_class_list(i,'CBM') <= capacity):
      result.append(i)

  return result


def ClassifyByAngle(OrderList) : # 각도가 유사한 조합들 분류 return : [(O1, O3), (O2,O1), ... ]
  resultList = []
  threshold = 45

  for tup in OrderList :
    angle1, angle2 = tup[0].angle, tup[1].angle
    if angle1 > angle2 :
      angle1, angle2 = angle2, angle1
    # print("angle1= ", angle1, "\t angle2 = ", angle2, "\t diff = ", angle2 - angle1)
    if angle2 - angle1 <= threshold or angle2 - angle1 >= 360 - threshold: # 각도의 차가 threshold보다 작거나 360 - threshold보다 큰 경우인 조합을 선택
      resultList.append(tup)

  return resultList


def ClassifyByDist(OrderList) : # 거리가 유사한 조합들 분류 return : [(O1, O3), (O2,O1), ... ]
  resultList = []
  threshold = 200

  for tup in OrderList :
    ter1, ter2 = tup[0].terminal, tup[1].terminal
    des1, des2 = tup[0].desID, tup[1].desID

    if Insoo[ter2, ter1][0] <= threshold and Insoo[des1, des2][0] <= threshold : # terminal 사이의 거리
      # print("Insoo[ter2, ter1][0] = ", Insoo[ter2, ter1][0])
      # print("Insoo[ter1, ter2][0] = ", Insoo[ter1, ter2][0])
      resultList.append(tup)

  return resultList

In [ ]:
def is_within_ranges(value, tuple_list):
    for start, end in tuple_list:
        if start <= value < end:
            return True
    return False

def canCompletePathInTime(path): # 도달 불가시 -1 반환, 그 외 경로의 거리 반환
  plustime = 0
  dist = 0
  start = path[0]

  dist += Insoo[path[0].id, path[1].id][0]
  plustime += Insoo[path[0].id, path[1].id][1]
  if not is_within_ranges(nowClock + plustime + Insoo[path[1].id, path[2].desID][1], path[2].timeWindow):
    return -1

  dist += Insoo[path[1].id, path[2].desID][0]
  plustime += Insoo[path[1].id, path[2].desID][1]

  if not (path[2].desID != path[3].desID):
    plustime += 60

  if not is_within_ranges(nowClock + plustime + Insoo[path[2].desID, path[3].desID][1], path[3].timeWindow):
    return -1

  dist += Insoo[path[2].desID, path[3].desID][0]
  plustime += Insoo[path[2].desID, path[3].desID][1]
  return dist

def makePerfectRoute(orderTuple): # 최적 경로를 반환. 없으면 [], 214957289(임의 값)를 반환
  perfectPath = list()
  perfectDist = 214957289

  terminalSeqs = [
        [gTerminals[orderTuple[0].terminal], gTerminals[orderTuple[1].terminal]],
        [gTerminals[orderTuple[1].terminal], gTerminals[orderTuple[0].terminal]]
      ]
  orderSeqs = [[orderTuple[0], orderTuple[1]], [orderTuple[1], orderTuple[0]]]
  for terminals in terminalSeqs:
    for orders in orderSeqs:
      tempPath = terminals + orders
      tempDist = canCompletePathInTime(tempPath)
      if (tempDist >= 0 and tempDist < perfectDist):
        perfectDist = tempDist
        perfectPath = tempPath

  return perfectPath, perfectDist

def perfectRouteChecking(orderTupleList): # 실행 함수. parameter: [(o1, o2), ...]
  ret = 0 # 처리된 경로 수
  visited = set() # 여기 주문 객체 값이 있으면 처리된 것임.

  # 효율성 테스트: 최적 경로 / sum(둘의 직선 거리) 값이 작을수록 좋음.
  # 경로는 첫번째 터미널에서 맨앞 활성 차량을 끌고 오거나 맨앞 비활성 차량을 끌고 온다. 아직 차량이 없는 경우는 생각하지 않는다.
  pathList = list() # 효율성 테스트에 따라 정렬할 리스트. ([경로], 효율성)

  for orderTuple in orderTupleList:
    perfectPath, perfectDist = makePerfectRoute(orderTuple)
    if len(perfectPath) == 0: continue

    sumOfStraightDist = Insoo[(orderTuple[0].terminal, orderTuple[0].desID)][0] + \
        Insoo[(orderTuple[1].terminal, orderTuple[1].desID)][0]
    pathList.append((perfectPath, perfectDist / sumOfStraightDist))

  pathList.sort(key=lambda x: x[1])
  print(pathList)
  for path in pathList:
    if path[0][2] in visited or path[0][3] in visited: continue
    if path[0][2].desID == path[0][3].desID: continue
    ret += 1
    visited.add(path[0][2])
    visited.add(path[0][3])

    curTerminal = path[0][0]
    curVeh = curTerminal.activeVehicleList[0] if len(curTerminal.activeVehicleList) != 0 \
                    else curTerminal.inactiveVehicleList[0] if len(curTerminal.inactiveVehicleList) != 0 \
                    else None
    if(curVeh == None) : continue
    curVeh.updateForDTDD(path[0])

  return ret

def difTerDifDest():
  totalOrderList = list()

  for terminalId, terminal in gTerminals.items():
    if terminal.orderNum == 0:
      continue
    totalOrderList.extend(flatten_dict_values(terminal.orderDict))

  result = CapacitiyLimitCombination4(totalOrderList)
  result = ClassifyByAngle(result)
  result = ClassifyByDist(result)
  result = perfectRouteChecking(result)

### 5번 알고리즘

In [ ]:
def process80():
  for terminalId, terminal in gTerminals.items():
    orderList = []
    for _, o in terminal.orderDict.items():
      orderList.extend(o)
    if len(orderList) == 0: return

    orderList.sort(key=lambda x: x.CBM, reverse=True)
    processCnt = len(orderList) * 80 // 100

    activeVehList = terminal.activeVehicleList.copy()
    inactiveVehList = terminal.inactiveVehicleList.copy()
    while len(activeVehList) > 0:
      if processCnt < 0:
        break

      curVeh = activeVehList[-1]
      start = 0
      end = processCnt
      while start <= end:
        mid = (start + end) // 2
        if curVeh.maxCBM < orderList[mid].CBM:
          start = mid + 1
        else:
          end = mid - 1

      if start <= processCnt and is_within_ranges(nowClock+orderList[start].time,orderList[start].timeWindow):
        curVeh.updateForUrgent(orderList[start])
        print("5번에서 처리된 것:", str(orderList[start]))
        orderList.remove(orderList[start])
        processCnt -= 1
      activeVehList.remove(curVeh)

    while len(inactiveVehList) > 0:
      if processCnt < 0:
        break

      curVeh = inactiveVehList[-1]
      start = 0
      end = processCnt
      while start <= end:
        mid = (start + end) // 2
        print(start, mid, end)
        if curVeh.maxCBM < orderList[mid].CBM:
          start = mid + 1
        else:
          end = mid - 1

      if start <= processCnt and is_within_ranges(nowClock+orderList[start].time,orderList[start].timeWindow):
        curVeh.updateForUrgent(orderList[start])
        print("5번에서 선택된 차량:", curVeh, str(curVeh))
        print("5번에서 처리된 것:", str(orderList[start]))
        orderList.remove(orderList[start])
        processCnt -= 1
      inactiveVehList.remove(curVeh)

In [ ]:
def processCBM20():
  for terminalId, terminal in gTerminals.items():
    orderList = []
    for _, o in terminal.orderDict.items():
      for realO in o:
        if realO.CBM <= 20 and is_within_ranges(nowClock+realO.time, realO.timeWindow):
          orderList.append(realO)

    orderList.sort(key=lambda x: x.CBM, reverse=True)
    processCnt = len(orderList) - 1

    activeVehList = terminal.activeVehicleList
    inactiveVehList = terminal.inactiveVehicleList
    while len(activeVehList) > 0:
      if processCnt < 0:
        break

      curVeh = activeVehList[-1]
      start = 0
      end = processCnt
      while start <= end:
        mid = (start + end) // 2
        if curVeh.maxCBM < orderList[mid].CBM:
          start = mid + 1
        else:
          end = mid - 1

      if start > processCnt:
        inactiveVehList.remove(curVeh)

      curVeh.updateForUrgent(orderList[start])
      orderList.remove(orderList[start])
      processCnt -= 1
    while len(inactiveVehList) > 0:
      if processCnt < 0:
        break

      curVeh = inactiveVehList[-1]
      start = 0
      end = processCnt
      while start <= end:
        mid = (start + end) // 2
        if curVeh.maxCBM < orderList[mid].CBM:
          start = mid + 1
        else:
          end = mid - 1

      if start > processCnt:
        inactiveVehList.remove(curVeh)

      curVeh.updateForUrgent(orderList[start])
      orderList.remove(orderList[start])
      processCnt -= 1

## 2-2.5) 통합 처리 알고리즘

In [ ]:
def totalRoutingAlgorithm():
  print("~~~~~~~~~~~~~1번 알고리즘 시작~~~~~~~~~~~~~~~~")
  sameTerSameDest()
  print("^^^^^^^^^^^^^^^2번 알고리즘 시작^^^^^^^^^^^^^^^")
  difTerSameDest()
  print("*****************3번 알고리즘 시작************")
  sameTerDifDest()
  print("%%%%%%%%%%%%%%%%%4번 알고리즘 시작%%%%%%%%%%%")
  difTerDifDest()
  print("%@%@%%#$%#$5번 시작#$^#^$%^$%^#&")
  process80()

## 2-3) 긴급 유보 처리 알고리즘

In [ ]:
def selectUrgentVehicle(terminal, cbm):
  aVeh = terminal.activeVehicleList
  bVeh = terminal.inactiveVehicleList
  # aVeh.sort(key=lambda x: x.maxCBM)
  start = 0
  end = len(aVeh) - 1
  #print(aVeh[0].id)
  while start <= end:
    mid = (start + end) // 2
    maxCBM = aVeh[mid].maxCBM
    if maxCBM >= cbm:
      start = mid + 1
    else: end = mid - 1

  if end != -1: return aVeh[end]

  start = 0
  end = len(bVeh) - 1
  # bVeh.sort(key=lambda x: x.maxCBM)
  #print(len(bVeh))
  while start <= end:
    mid = (start + end) // 2
    #print(mid)
    maxCBM = bVeh[mid].maxCBM
    if maxCBM >= cbm:
      start = mid + 1
    else: end = mid - 1

  if end != -1: return bVeh[end]

  return None

In [ ]:
def urgentDelivery(order):
  curTerminal = gTerminals[order.terminal]
  curVeh = selectUrgentVehicle(curTerminal, order.CBM)
  curVeh.updateForUrgent(order)

## 2-4) 차량 복귀 알고리즘

In [ ]:
tT = []
for terminal in gTerminals.values():
  tT.append(terminal)

def findBestReturnTer(vehicle) :
  global tT, nowClock

  if nowClock < 7000:
    selectedTerminal = sorted(tT, key = lambda ter: (ter.currVehNum + ter.expVehNum) * Insoo[(vehicle.nowLoc,ter.id)][0] / max(ter.orderNum,0.001))[0]
  else:
    selectedTerminal = sorted(tT, key = lambda ter: (ter.currVehNum + ter.expVehNum + len(ter.inactiveVehicleList)) / max(ter.orderNum,0.001))[0]

  return (nowClock + Insoo[vehicle.nowLoc,selectedTerminal.id][1],selectedTerminal)

## 2-5) 결과 출력 테이블

In [ ]:
def createOrderResults():
    global orderResults, nowClock
    Clock = timestampToDateStr(nowClock)
    result = orderResults.copy()
    indices_to_remove = []

    for i in result.index:
        if result.at[i,"ArrivalTime"] < Clock:
          if result.at[i,"DepartureTime"] < Clock:
            continue
          elif "D" in result.at[i,"SiteCode"]:
              result.at[i, 'ArrivalTime'] = 'Null'
              result.at[i, 'WaitingTime'] = 'Null'
              result.at[i, 'ServiceTime'] = 'Null'
              result.at[i, 'DepartureTime'] = 'Null'
              result.at[i, 'Delivered'] = 'No'
          else:
            indices_to_remove.append(i)
        else:
            indices_to_remove.append(i)

    result.drop(indices_to_remove, inplace=True)

    print(result)

In [ ]:
def createVehicleResults() :
  global vehicleResults
  for i in range(1125):
    vehicle = globals()["VEH_"+str(i+2)]
    if vehicle.isActive :
      variableCost = vehicle.totalMoveTime * vehicle.variableCost
      vehicleResults = pd.concat([vehicleResults, pd.DataFrame({'VehNum': [vehicle.id],'Count': [len(vehicle.processed)],\
                                  'Volume':[sum_variable_in_class_list(vehicle.processed, 'CBM')],'TravelDistance':[vehicle.totalDistance],\
                                  'WorkTime':[vehicle.totalMoveTime + vehicle.totalUnloadedTime],'TravelTime':[vehicle.totalMoveTime],\
                                  'ServiceTime':[vehicle.totalUnloadedTime],'WaitingTime':[vehicle.totalWaitingTime],'TotalCost':[variableCost + vehicle.fixedCost],\
                                  'FixedCost':[vehicle.fixedCost],'VariableCost':[variableCost]})],\
                                             ignore_index=True)

  print(vehicleResults)

# 3. 시뮬레이션 구현

- Timestamp

- 0 orderByTime : 주문 시점을 나타내는 timestamp

- 1 departDestination : 최종 하역 작업 완료

- 2 imBack : 차량 터미널 복귀

- 3 deadline : 유보 처리 알고리즘

- 4 recoverStarvation : 차량 강제 분배 알고리즘

- 5 end : 주문 종료 sequence 실행

## A. Event 함수

#### initialize : 초기화 / end : 결과 출력 알고리즘

In [ ]:
def initialize():
  global nowClock, deadlineList, orderbytime, timestamp
  timestamp = [360.0,20000.0,20000.0,20000.0,60.0,10080.0]
  nowClock = 0

  # 초기 터미널의 inactiveVehicle들의 차량 정렬 > 차량을 큰 순서대로 쓸 것이기 때문에 초기 정렬 시 차량 복귀 외 정렬 필요x
  for terminalId, terminal in gTerminals.items():
    terminal.inactiveVehicleList.sort(key=lambda x : x.maxCBM, reverse=True)

  for o in orderbytime[0]:
    if (gTerminals[o.terminal].orderDict.get(o.desID) is None):
      gTerminals[o.terminal].orderDict[o.desID] = []
    gTerminals[o.terminal].orderDict[o.desID].append(o)
    gTerminals[o.terminal].orderNum += 1
    print(type(o.deadline))
    deadlineList.put((o.deadline,o))

  if (deadlineList.qsize() != 0):
    top = deadlineList.get()
    timestamp[3] = top[0]
    deadlineList.put(top)

  totalRoutingAlgorithm()

  if (unloadedComplete.qsize() != 0):
    top = unloadedComplete.get()
    timestamp[1] = top[0]
    unloadedComplete.put(top)

#### timing : timestamp 내에서 가장 빠른 시간대의 Event를 찾아서 type을 반환해주는 함수

In [ ]:
def timing():
  global timestamp, nowClock
  min_value = min(timestamp)                ## timestamp에서 가장 짧은 시간대를 찾기
  nowClock = min_value                      ## 그 시간을 nowClock에 update => 해당 event의 실행으로 시계를 이동시킴
  event_type = timestamp.index(min_value)   ## 실행할 event의 index를 받아오기
  return event_type

#### OrderByTime : Group 시간대에 해당하는 주문들을 생성하는 함수


In [ ]:
# orderbytime에 있는 주문들을 해당 시간이 되면 각 terminal객체의 orderDict에 넣어주는 함수
def UpdateTerByOrderDesID(List):
  global gTerminals, deadlineList
  for order in List :
    # print(gTerminals[order.terminal], order.terminal, order.desID)
    if gTerminals[order.terminal].orderDict.get(order.desID, 0) == 0 :
      gTerminals[order.terminal].orderDict[order.desID] = []
    gTerminals[order.terminal].orderDict[order.desID].append(order)
    gTerminals[order.terminal].orderNum += 1                                #주문 객체를 추가하면서 해당 터미널의 현재 주문 수(orderNum) 증가
    deadlineList.put((order.deadline,order))


In [ ]:
# 메인 함수
def orderByTime():
  global timestamp, deadlineList, orderbytime
  group = timestamp[0]/360.0
  timestamp[0] += 360.0
  if timestamp[0] == 8640: timestamp[0] = 20000.0

  UpdateTerByOrderDesID(orderbytime[group])
  if (deadlineList.qsize() != 0):
    top = deadlineList.get()
    timestamp[3] = top[0]
    deadlineList.put(top)

  totalRoutingAlgorithm()
  if (unloadedComplete.qsize() != 0):
    top = unloadedComplete.get()
    timestamp[1] = top[0]
    unloadedComplete.put(top)

  createOrderResults()

#### departDestination : 작업완료 후 목적지에서 복귀 터미널을 결정하는 알고리즘


- ((목적지 작업완료 시간, 차량객체),(목적지 작업 완료시간,차량객체))
- findBestReturnTer => return 터미널 도착시간, 터미널 객체

In [ ]:
def departDestination():
  global unloadedComplete, timestamp, headToTer
  selectedVeh = unloadedComplete.get()[1]
  returnTime, returnTerminal = findBestReturnTer(selectedVeh)
  if returnTime < 10080:                                      # 복귀 명령을 내려도 의미없을 때 = 복귀해도 시간상 주문 처리가 안될 때
    returnTerminal.expVehNum += 1
    headToTer.put((returnTime, returnTerminal, selectedVeh))

  top = headToTer.get()
  timestamp[2] = top[0]
  headToTer.put(top)

  if (unloadedComplete.qsize() != 0):
    top = unloadedComplete.get()
    timestamp[1] = top[0]
    unloadedComplete.put(top)
  else:
    timestamp[1] = 20000.0

#### imBack : 터미널 복귀 알고리즘

In [ ]:
def generalRoutingAlgorithm(topVeh):
  global nowClock

  curTerminal = gTerminals[topVeh.nowLoc]

  if curTerminal.orderNum == 0: return

  selectedOrder = None
  difference = 100
  for dest,orderList in curTerminal.orderDict.items():
    for order in orderList:
      if (topVeh.maxCBM >= order.CBM) and (difference > topVeh.maxCBM - order.CBM) and (is_within_ranges(nowClock+order.time,order.timeWindow)):
        selectedOrder = order
        difference = topVeh.maxCBM - order.CBM
  if selectedOrder == None : return
  topVeh.updateForUrgent(selectedOrder)

In [ ]:
def imBack():
  global headToTer, timestamp
  top = headToTer.get()
  topTime, headTer, topVeh = top[0], top[1], top[2]
  # 차량 객체의 내부 정보 update
  topVeh.totalDistance += Insoo[topVeh.nowLoc,headTer.id][0]
  topVeh.totalMoveTime += Insoo[topVeh.nowLoc,headTer.id][1]
  topVeh.nowLoc = headTer.id
  topVeh.terminalReturnTime = topTime
  # 터미널 객체의 내부 정보 update
  headTer.expVehNum -= 1
  headTer.currVehNum += 1
  headTer.activeVehicleList.append(topVeh)
  headTer.activeVehicleList.sort(key=lambda x : x.maxCBM, reverse=True)

  if(headToTer.qsize() != 0):
    top = headToTer.get()
    timestamp[2] = top[0]
    headToTer.put(top)
  else:
    timestamp[2] = 20000

  generalRoutingAlgorithm(topVeh)

  if (unloadedComplete.qsize() != 0):
    top = unloadedComplete.get()
    timestamp[1] = top[0]
    unloadedComplete.put(top)

#### deadline : 긴급 유보 처리 알고리즘

In [ ]:
def processDeadline():
  global timestamp, deadlineList

  if (deadlineList.qsize() == 0):
    timestamp[3] = 20000
    return

  top = deadlineList.get()[1]
  if not top.delivered:
    urgentDelivery(top)
    if(deadlineList.qsize() != 0):
      t = deadlineList.get()
      timestamp[3] = t[0]
      deadlineList.put(t)

  else:
    if(deadlineList.qsize() != 0):
      t = deadlineList.get()
      timestamp[3] = t[0]
      deadlineList.put(t)

#### recoverStarvation : 차량 강제 분배 알고리즘

In [ ]:
def recoverStarvation():
  global tT, nowClock, timestamp
  if nowClock <= 5080:
    timestamp[4] += 60.0
  else:
    timestamp[4] += 5.0

  if timestamp[4] == 10080: timestamp[4] = 20000.0

  sortedTerminals = None

  if nowClock < 4000:
    sortedTerminals = sorted(tT, key = lambda ter: (ter.currVehNum + ter.expVehNum) / max(ter.orderNum,0.001))
  else:
    sortedTerminals = sorted(tT, key = lambda ter: (ter.currVehNum + ter.expVehNum + len(ter.inactiveVehicleList)) / max(ter.orderNum,0.001))

    top1Ter = sortedTerminals[0]
    sortedTerminals = sorted(tT, key = lambda ter: len(ter.activeVehicleList), reverse=True)

    selectedTerminal = None
    if len(sortedTerminals[4].activeVehicleList) != 0:
      selectedTerminal = sortedTerminals[0]
    else:
      maxInactiveIdx = 0
      maxVal = -1
      for i in range(len(sortedTerminals)):
        if len(sortedTerminals[i].inactiveVehicleList) > maxVal:
          maxVal = len(sortedTerminals[i])
          maxInactiveIdx = i
      selectedTerminal = sortedTerminals[maxInactiveIdx]

    curVeh = None # None으로 유지되면 차량 부족으로 인한 긴급 배송 불가
    if len(selectedTerminal.activeVehicleList) > 0:
      curVeh = selectedTerminal.activeVehicleList[0] # CBM 큰 순
    else:
      curVeh = selectedTerminal.inactiveVehicleList[0] # CBM 큰 순
    curVeh.updateForStarvation(top1Ter) # 차량 보냄

#### end : 종료 알고리즘

In [ ]:
def end():
  createOrderResults()
  createVehicleResults()

## B. 시뮬레이션 실행

### 전체 구동

In [ ]:
initialize()

while (nowClock <= 10080):
  event_type = timing()

  if event_type == 0:
    orderByTime()
  elif event_type == 1:
    departDestination()
  elif event_type == 2:
    imBack()
  elif event_type == 3:
    processDeadline()
  elif event_type == 4:
    recoverStarvation()
  else:
    end()
    break

Streaming output truncated to the last 5000 lines.
possibleCombination...
가능한 경로 조합3
[]
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
----------------------------------
현재 터미널: O_349
CapacitiyLimitCombination...
possibleCombination...
가능한 경로 조합3
[]
1440.0
1440.0
----------------------------------
현재 터미널: O_352
CapacitiyLimitCombination...
possibleCombination...
가능한 경로 조합3
[]
1440.0
----------------------------------
현재 터미널: O_376
CapacitiyLimitCombination...
possibleCombination...
가능한 경로 조합3
[]
1440.0
1440.0
1440.0
1440.0
----------------------------------
현재 터미널: O_430
CapacitiyLimitCombination...
possibleCombination...
가능한 경로 조합3
[]
1440.0
----------------------------------
현재 터미널: O_448
CapacitiyLimitCombination...
possibleCombination...
가능한 경로 조합3
[]
1440.0
1440.0
1440.0
1440.0
1440.0
1440.0
----------------------------------
현재 터미널: O_466
CapacitiyLimitCombination...
possibleCombination...
가능한 경로 조합3
[]
1440.0
-------------------

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



%@%@%%#$%#$5번 시작#$^#^$%^$%^#&
0 21 43
22 32 43
22 26 31
22 23 25
24 24 25
25 25 25
0 21 43
22 32 43
22 26 31
22 23 25
24 24 25
25 25 25
0 21 43
22 32 43
22 26 31
22 23 25
24 24 25
25 25 25
0 21 43
22 32 43
22 26 31
22 23 25
24 24 25
25 25 25
0 21 43
22 32 43
22 26 31
22 23 25
24 24 25
25 25 25
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
0 1 3
0 0 0
               ORD_NO VehicleID Sequence SiteCode       ArrivalTime  \
0                Null    VEH_48        1      O_9  2023-05-01 00:00   
1           M18434024    VEH_48        2    D_301  2023-05-01 00:22   
2           M18434020    VEH_48        3    D_301  2023-05-01 00:22   
3                Null    VEH_47        1      O_9  2023-05-01 00:00   
4     MF2305036201264    VEH_47        2   D_1180  2023-05-01 

#### 결과 확인

In [ ]:
timestamp

In [ ]:
orderResults

In [ ]:
vehicleResults

In [ ]:
unloadedComplete.qsize(), headToTer.qsize()

In [ ]:
orderResults[orderResults['ORD_NO'] != 'Null']

#### 결과 csv로 저장

In [ ]:
orderResults.to_csv("/content/drive/Shareddrives/CJ 대한통운 공모전/Final/TotalOrderResults.csv")

In [ ]:
vehicleResults.to_csv("/content/drive/Shareddrives/CJ 대한통운 공모전/Final/TotalVehicleResults.csv")

In [ ]:
def createOrderResultsByGroup(time):
    global orderResults
    Clock = timestampToDateStr(time*360.0)
    result = orderResults.copy()
    indices_to_remove = []

    for i in result.index:
        if result.at[i,"ArrivalTime"] < Clock:
          if result.at[i,"DepartureTime"] < Clock:
            continue
          elif "D" in result.at[i,"SiteCode"]:
              result.at[i, 'ArrivalTime'] = 'Null'
              result.at[i, 'WaitingTime'] = 'Null'
              result.at[i, 'ServiceTime'] = 'Null'
              result.at[i, 'DepartureTime'] = 'Null'
              result.at[i, 'Delivered'] = 'No'
          else:
            indices_to_remove.append(i)
        else:
            indices_to_remove.append(i)

    result.drop(indices_to_remove, inplace=True)

    return result

In [ ]:
for i in range(1,24):
  globals()["Table"+str(i)] = createOrderResultsByGroup(i)
  globals()["Table"+str(i)].to_csv(f"/content/drive/Shareddrives/CJ 대한통운 공모전/Final/OrderResultsFolder/OrderResults{i}.csv")

#### 주문 조회

In [ ]:
orderid = input("주문번호를 입력하세요 : ")
order[order["OrderID"]==orderid][["OrderID","CBM","Departure","Des_ID","Unloaded_start","Unloaded_end","distance","time","group"]]

In [ ]:
len(order[["Des_ID"]].drop_duplicates())

In [ ]:
Insoo["O_11","O_179"]

1:45 => 5시간 19분 6시 58분

In [ ]:
print_class(O590)

In [ ]:
print_class(VEH_48)

In [ ]:
print(f"처리해야 하는 주문 건 수 : {len(order)} / 출력된 처리 건수 : {len(orderResults[orderResults['ORD_NO'] != 'Null'])}")
# orderResults[orderResults['ORD_NO'] != 'Null']
# order

In [ ]:
orderResults[orderResults['ORD_NO'] != 'Null']

In [ ]:
order

미처리된 주문 처리 확인

In [ ]:
for i in range(15617):
  order_class = globals()["O"+str(i)]
  if not order_class.delivered:
    print_class(order_class) # 미처리된 주문 class의 상세 정보 출력

#### 대쉬보드 확인

In [ ]:
for terminalid, terminal in gTerminals.items():
  print(f"터미널 {terminalid}에 남아 있는 주문 수 : {len(flatten_dict_values(terminal.orderDict))}")
  for dest, orderlist in terminal.orderDict.items():
    print(f"목적지 {dest}에 남아 있는 주문 수 : {len(orderlist)}")
  print(" ")

In [ ]:
active_sum = 0
for terminalid, terminal in gTerminals.items():
  active_sum += len(terminal.activeVehicleList)
  print(f"{terminalid}의 활성화 차량 대수 : {len(terminal.activeVehicleList)}, 비활성화 차량 대수 : {len(terminal.inactiveVehicleList)}")
print(f"사용한 총 차량 대수 : {active_sum+headToTer.qsize()+unloadedComplete.qsize()}")

#### 기초 통계

In [ ]:
order.groupby("Departure").count()